# Feature Engineering

In [121]:
import pandas as pd
import numpy as np

In [122]:
pd.set_option("display.max_columns", None)

In [123]:
all_games = pd.read_csv("data/games_clean.csv", index_col="id")

In [124]:
all_games.head()

,date,home_team_score,period,postseason,season,status,visitor_team_score,home_team.id,home_team.abbreviation,home_team.conference,home_team.division,home_team.full_name,visitor_team.id,visitor_team.abbreviation,visitor_team.conference,visitor_team.division,visitor_team.full_name
id,,,,,,,,,,,,,,,,,
47179,2019-01-30T00:00:00.000Z,126,4,False,2018,Final,94,2,BOS,East,Atlantic,Boston Celtics,4,CHA,East,Southeast,Charlotte Hornets
48751,2019-02-09T00:00:00.000Z,112,4,False,2018,Final,123,2,BOS,East,Atlantic,Boston Celtics,13,LAC,West,Pacific,LA Clippers
48739,2019-02-08T00:00:00.000Z,117,4,False,2018,Final,110,23,PHI,East,Atlantic,Philadelphia 76ers,8,DEN,West,Northwest,Denver Nuggets
48740,2019-02-08T00:00:00.000Z,119,4,False,2018,Final,106,30,WAS,East,Southeast,Washington Wizards,6,CLE,East,Central,Cleveland Cavaliers
48746,2019-02-08T00:00:00.000Z,102,4,False,2018,Final,96,26,SAC,West,Pacific,Sacramento Kings,16,MIA,East,Southeast,Miami Heat


In [128]:
# Feature engineering

g = all_games.copy()

g["date"] = pd.to_datetime(games["date"]).dt.tz_localize(None)

# creating target variable
g["winner"] = np.where(g["home_team_score"] > g["visitor_team_score"], 1, 0)

# creating avg points by team across all years (not sure how useful this will be)
home_avg_pts_map = g[["home_team.id", "home_team_score"]].groupby("home_team.id").mean().squeeze()
g["home_team_avg_score_historical"] = g["home_team.id"].map(home_avg_pts_map).round(1)

visitor_avg_pts_map = g[["visitor_team.id", "visitor_team_score"]].groupby("visitor_team.id").mean().squeeze()
g["visitor_team_avg_score_historical"] = g["visitor_team.id"].map(visitor_avg_pts_map).round(1)

In [205]:
g["home_team_id_year"] = (g["home_team.id"].astype(str) + " " + g["season"].astype(str)).values
g["visitor_team_id_year"] = (g["visitor_team.id"].astype(str) + " " + g["season"].astype(str)).values

In [206]:
home_avg_score_map = g[["home_team_id_year", "home_team_score"]].groupby(["home_team_id_year"]).mean().squeeze()
home_avg_score_map

home_team_id_year
1 1979    106.953488
1 1980    107.634146
1 1981    101.714286
1 1982    103.214286
1 1983    103.790698
             ...    
9 2016    102.975610
9 2017    105.731707
9 2018    108.418605
9 2019    109.303030
9 2020    105.083333
Name: home_team_score, Length: 1164, dtype: float64

In [207]:
visitor_avg_score_map = g[["visitor_team_id_year", "visitor_team_score"]].groupby(["visitor_team_id_year"]).mean().squeeze()
visitor_avg_score_map

visitor_team_id_year
1 1979    101.318182
1 1980    102.219512
1 1981     99.523810
1 1982     97.279070
1 1983     98.318182
             ...    
9 2016     99.682927
9 2017    101.804878
9 2018    104.837209
9 2019    105.181818
9 2020    108.138889
Name: visitor_team_score, Length: 1164, dtype: float64

In [210]:
g["home_team_avg_score"] = g["home_team_id_year"].map(home_avg_score_map).round(1)
g["visitor_team_avg_score"] = g["visitor_team_id_year"].map(visitor_avg_score_map).round(1) 

In [211]:
g

,date,home_team_score,period,postseason,season,status,visitor_team_score,home_team.id,home_team.abbreviation,home_team.conference,home_team.division,home_team.full_name,visitor_team.id,visitor_team.abbreviation,visitor_team.conference,visitor_team.division,visitor_team.full_name,winner,home_team_avg_score_historical,visitor_team_avg_score_historical,home_team_id_year,visitor_team_id_year,home_team_avg_score,visitor_team_avg_score
id,,,,,,,,,,,,,,,,,,,,,,,,
47179,2019-01-30,126,4,False,2018,Final,94,2,BOS,East,Atlantic,Boston Celtics,4,CHA,East,Southeast,Charlotte Hornets,1,105.8,98.2,2 2018,4 2018,112.8,108.3
48751,2019-02-09,112,4,False,2018,Final,123,2,BOS,East,Atlantic,Boston Celtics,13,LAC,West,Pacific,LA Clippers,0,105.8,100.7,2 2018,13 2018,112.8,113.1
48739,2019-02-08,117,4,False,2018,Final,110,23,PHI,East,Atlantic,Philadelphia 76ers,8,DEN,West,Northwest,Denver Nuggets,1,103.3,104.2,23 2018,8 2018,117.9,108.2
48740,2019-02-08,119,4,False,2018,Final,106,30,WAS,East,Southeast,Washington Wizards,6,CLE,East,Central,Cleveland Cavaliers,1,103.4,98.3,30 2018,6 2018,116.4,103.8
48746,2019-02-08,102,4,False,2018,Final,96,26,SAC,West,Pacific,Sacramento Kings,16,MIA,East,Southeast,Miami Heat,1,105.5,96.0,26 2018,16 2018,114.9,105.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128052,2021-03-03,111,4,False,2020,Final,114,6,CLE,East,Central,Cleveland Cavaliers,12,IND,East,Central,Indiana Pacers,0,102.3,100.1,6 2020,12 2020,107.5,117.7
128036,2021-03-01,124,4,False,2020,Final,130,22,ORL,East,Southeast,Orlando Magic,7,DAL,West,Southwest,Dallas Mavericks,0,101.9,101.5,22 2020,7 2020,105.4,113.8
128001,2021-02-25,111,4,False,2020,Final,97,23,PHI,East,Atlantic,Philadelphia 76ers,7,DAL,West,Southwest,Dallas Mavericks,1,103.3,101.5,23 2020,7 2020,117.3,113.8


In [212]:
g.to_csv("data/games_with_features.csv")

In [86]:
feat_names = ["season", "winner", "home_team_avg_score", "visitor_team_avg_score"]
features = g.loc[:, feat_names]

features.to_csv("data/features.csv")

### Using only 2019 season

In [95]:
# Feature engineering

g = all_games[all_games["season"].eq(2019)].copy()

g["date"] = pd.to_datetime(games["date"]).dt.tz_localize(None)

g["winner"] = np.where(g["home_team_score"] > g["visitor_team_score"], 1, 0)

home_avg_pts_map = g[["home_team.id", "home_team_score"]].groupby("home_team.id").mean().squeeze()
g["home_team_avg_score"] = g["home_team.id"].map(home_avg_pts_map)
g["home_team_avg_score"] = g["home_team_avg_score"].round(1)

visitor_avg_pts_map = g[["visitor_team.id", "visitor_team_score"]].groupby("visitor_team.id").mean().squeeze()
g["visitor_team_avg_score"] = g["visitor_team.id"].map(visitor_avg_pts_map)
g["visitor_team_avg_score"] = g["visitor_team_avg_score"].round(1)

In [97]:
feat_names = ["season", "winner", "home_team_avg_score", "visitor_team_avg_score"]
features = g.loc[:, feat_names]

In [99]:
features.to_csv("data/features_2019.csv")